# Gathering of Bayesian evidence for the ELM model

In [1]:
import pickle
from pathlib import Path

import numpy as np
from matplotlib import pyplot as plt
from periodictable import elements
from tqdm import tqdm

In [2]:
import elm

Using database version X4-2024-12-31 located in: /home/beyerk/db/exfor/unpack_exfor-2024/X4-2024-12-31


In [3]:
import exfor_tools
import jitr

In [4]:
proton = (1, 1)
neutron = (1, 0)

In [5]:
Ca40 = (40, 20)
Ca48 = (48, 20)

Zr90 = (90, 40)
Zr92 = (92, 40)
Zr94 = (94, 40)
Zr96 = (96, 40)

Sn124 = (124, 50)
Sn122 = (122, 50)
Sn120 = (120, 50)
Sn118 = (118, 50)
Sn116 = (116, 50)

Ba138 = (138, 56)

Sm144 = (144, 62)
Sm148 = (148, 62)

Pb208 = (208, 82)
Pb206 = (206, 82)

In [6]:
targets = [
    Ca40,
    Ca48,
    Zr90,
    Zr92,
    Zr94,
    Zr96,
    Sn116,
    Sn118,
    Sn120,
    Sn122,
    Sn124,
    Ba138,
    Sm144,
    Sm148,
    Pb206,
    Pb208,
]

In [7]:
workspaces_n_elastic = {}
workspaces_p_elastic = {}

In [8]:
core_solver = jitr.rmatrix.Solver(40)
channel_radius = 10 * np.pi
lmax = 20
theta_vis = np.linspace(0.1, np.pi, 180)

In [17]:
data_sets_by_isotope = {}
for (A,Z) in targets:
    fname = Path(f"{elements[Z]}{A}_diff_xs.pkl")
    if not fname.is_file():
        print(f"Could not find {fname}")
    else:
        with open(fname, "rb") as f:
            d = pickle.load(f)
            data_sets_by_isotope[(A,Z)] = d

In [18]:
data_sets_by_entry = {}
for (A,Z) in targets:
    d = data_sets_by_isotope[(A,Z)]
    for k,v in d['n,n'].items():
        if k in data_sets_by_entry:
            data_sets_by_entry[k].append(v)
        else:
            data_sets_by_entry[k] = [v]
    
    for k,v in d['p,p']['ratio'].items():
        if k in data_sets_by_entry:
            data_sets_by_entry[k].append(v)
        else:
            data_sets_by_entry[k] = [v]
    
    for k,v in d['p,p']['absolute'].items():
        if k in data_sets_by_entry:
            data_sets_by_entry[k].append(v)
        else:
            data_sets_by_entry[k] = [v]


In [19]:
def remove_duplicates(entries_ppr, entries_pp):
    all_dup = []
    for kr,vr in entries_ppr.items():
        for k,v in entries_pp.items():
            if kr == k:
                Eratio = [er.Einc for er in vr.measurements]
                nm = [e for e in v.measurements if e.Einc not in Eratio]
                if nm != v.measurements:
                    print(f"Found duplicates between (p,p) absolute and ratio to Rutherford in {k}")
                    print([x.Einc for x in v.measurements])
                    print([x.Einc for x in vr.measurements])

                    
                    # replace (p,p) absolute with only non-duplicate measurements
                    v.measurements = nm
                    if nm == []:
                        all_dup.append(k)
    
    # in cases where all (p,p) absolute measurements are duplicates, remove whole entry
    for k in all_dup:
        del entries_pp[k]

    return entries_ppr, entries_pp

In [21]:
for (A,Z) in targets:
    remove_duplicates(data_sets_by_isotope[(A,Z)]['p,p']['ratio'], data_sets_by_isotope[(A,Z)]['p,p']['absolute'])

In [9]:
def set_up_p_elastic_workspaces(target, entries_xs, entries_ratio):
    meas_ratio = exfor_tools.sort_measurements_by_energy(entries_ratio)
    meas_xs = exfor_tools.sort_measurements_by_energy(entries_xs)

    # only take non-ratio (p,p) data if it's not duplicate to ruth
    for i in range(len(entries_xs)):
        for j in range(len(entries_ratio)):
            if entries_xs[i].entry == entries_ratio[j].entry
            print(entries_xs[i].entry)
            
    workspaces = []
    for m in meas_ratio:
        workspaces.append(
            elm.DifferentialXS(
                proton,
                target,
                m.Elab,
                m,
                theta_vis,
                core_solver,
                channel_radius,
                lmax,
                absolute_xs=False,
            )
        )
    for m in meas_xs:
        workspaces.append(
            elm.DifferentialXS(
                proton,
                target,
                m.Elab,
                m,
                theta_vis,
                core_solver,
                channel_radius,
                lmax,
                absolute_xs=True,
            )
        )

    workspaces.sort(key=lambda w: w.exp.Elab)

    return workspaces

In [10]:
def set_up_n_elastic_workspaces(target, entries):
    measurements = exfor_tools.sort_measurements_by_energy(entries)

    return [
        elm.DifferentialXS(
            neutron, target, m.Elab, m, theta_vis, core_solver, channel_radius, lmax
        )
        for m in measurements
    ]

In [11]:
for t in targets:
    A, Z = t
    n_data_path = Path(f"./{elements[Z]}{A}_n_elastic.pkl")
    if n_data_path.is_file():
        with open(n_data_path, "rb") as f:
            entries = pickle.load(f)
            n_entries = len(entries)
            n_measurements = int(
                np.sum([len(entry.measurements) for entry in entries.values()])
            )
        print(
            f"Read {n_measurements} measurements from {n_entries} entries from {n_data_path}"
        )
        ws = set_up_n_elastic_workspaces(t, entries)
        workspaces_n_elastic[t] = ws
    p_data_path = Path(f"./{elements[Z]}{A}_p_elastic.pkl")
    if p_data_path.is_file():
        with open(p_data_path, "rb") as f:
            entries = pickle.load(f)
            n_entries = len(entries["xs"]) + len(entries["ratio"])
            n_measurements = int(
                np.sum([len(entry.measurements) for entry in entries["xs"].values()])
                + np.sum(
                    [len(entry.measurements) for entry in entries["ratio"].values()]
                )
            )
        print(
            f"Read {n_measurements} measurements from {n_entries} entries from {p_data_path}"
        )
        ws = set_up_p_elastic_workspaces(t, entries["xs"], entries["ratio"])
        workspaces_p_elastic[t] = ws

Read 12 measurements from 3 entries from Ca40_n_elastic.pkl
Read 43 measurements from 23 entries from Ca40_p_elastic.pkl
Read 2 measurements from 1 entries from Ca48_n_elastic.pkl
Read 15 measurements from 7 entries from Ca48_p_elastic.pkl
Read 3 measurements from 2 entries from Zr90_n_elastic.pkl
Read 23 measurements from 21 entries from Zr90_p_elastic.pkl
Read 2 measurements from 1 entries from Zr92_n_elastic.pkl
Read 9 measurements from 9 entries from Zr92_p_elastic.pkl
Read 2 measurements from 1 entries from Zr94_n_elastic.pkl
Read 5 measurements from 5 entries from Zr94_p_elastic.pkl
Read 2 measurements from 2 entries from Zr96_p_elastic.pkl
Read 3 measurements from 2 entries from Sn116_n_elastic.pkl
Read 7 measurements from 7 entries from Sn116_p_elastic.pkl
Read 2 measurements from 1 entries from Sn118_n_elastic.pkl
Read 1 measurements from 1 entries from Sn118_p_elastic.pkl
Read 3 measurements from 2 entries from Sn120_n_elastic.pkl
Read 11 measurements from 10 entries from Sn1

In [13]:
with open("../workspaces/corpus_elastic.pkl", "wb") as f:
    pickle.dump( {neutron : workspaces_n_elastic, proton : workspaces_p_elastic}, f)

In [17]:
np.sum([ len(l) for l in workspaces_n_elastic.values()])

np.int64(54)

In [18]:
np.sum([ len(l) for l in workspaces_p_elastic.values()])

np.int64(137)

In [29]:
np.sum([ np.sum([  ws.exp.data.shape[1] for ws in iso ]) for iso in workspaces_n_elastic.values() ])

np.float64(1285.0)

In [30]:
np.sum([ np.sum([  ws.exp.data.shape[1] for ws in iso ]) for iso in workspaces_p_elastic.values() ])

np.int64(5830)